In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import KFold

In [2]:
from create_matrix import *

from tensor_custom_core import *
from create_matrix import *
from tensor_custom_core import *
from degree_days import dds
appliance_index = {appliance: APPLIANCES_ORDER.index(appliance) for appliance in APPLIANCES_ORDER}

APPLIANCES = ['fridge', 'hvac', 'wm', 'mw', 'oven', 'dw']
region = "SanDiego"
year = 2014

import os


def un_normalize(x, maximum, minimum):
    return (maximum - minimum) * x + minimum

In [3]:
n_splits = 10
import sys
case=2
a=2
cost='abs'

In [4]:
def get_tensor(df, dfc):
    start, stop = 1, 13
    energy_cols = np.array(
        [['%s_%d' % (appliance, month) for month in range(start, stop)] for appliance in APPLIANCES_ORDER]).flatten()

    static_cols = ['area', 'total_occupants', 'num_rooms']
    static_df = df[static_cols]
    static_df = static_df.div(static_df.max())
    weather_values = np.array(dds[2014][region][start - 1:stop - 1]).reshape(-1, 1)

    dfc = df.copy()

    df = dfc[energy_cols]
    col_max = df.max().max()
    col_min = df.min().min()
    # df = (1.0 * (df - col_min)) / (col_max - col_min)
    tensor = df.values.reshape((len(df), 7, stop - start))
    M, N, O = tensor.shape
    return tensor

In [5]:
df, dfc = create_matrix_single_region("SanDiego", year)
tensor = get_tensor(df, dfc)

In [6]:
from sklearn.model_selection import train_test_split, KFold

In [7]:
n_splits = 10


In [8]:
from common import compute_rmse

In [9]:
np.arange(0.1, 1.0, 0.1)

array([ 0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9])

In [15]:
pred = {}
sd = {}
out = {}
n_splits = 10
case = 2
a = 2
cost = 'abs'
for random_seed in range(2):
    pred[random_seed] = {}
    for appliance in APPLIANCES_ORDER:
        pred[random_seed][appliance] = {f:[] for f in range(70, 110, 10)}

kf = KFold(n_splits=n_splits)
for random_seed in range(2):
    print(random_seed,"\n")
    for train_percentage in range(70, 110, 10):
        print train_percentage, "\n"
        for train_max, test in kf.split(df):

            num_train = int((train_percentage*len(train_max)/100)+0.5)
            if train_percentage==100:
                train = train_max
            else:
                train, _ = train_test_split(train_max, train_size = train_percentage/100.0, random_state=random_seed)
            train_ix = df.index[train]
            test_ix = df.index[test]
            print "train_ix"
            print train_ix
            
            num_test = len(test_ix)
            train_test_ix = np.concatenate([test_ix, train_ix])
            df_t, dfc_t = df.ix[train_test_ix], dfc.ix[train_test_ix]
            tensor = get_tensor(df_t, dfc_t)
            tensor_copy = tensor.copy()
            # First n
            tensor_copy[:num_test, 1:, :] = np.NaN
            H, A, T = learn_HAT(case, tensor_copy, a, a, num_iter=2000, lr=0.1, dis=False, cost_function=cost)
            HAT = multiply_case(H, A, T, case)
            for appliance in APPLIANCES_ORDER:
                pred[random_seed][appliance][train_percentage].append(pd.DataFrame(HAT[:num_test, appliance_index[appliance], :], index=test_ix))


(0, '\n')
70 

train_ix
Int64Index([9836, 9612, 8342, 9370, 4083, 4934, 3687, 6377, 5909, 9585, 7639,
            1731, 4761, 3864, 7409, 2606, 6429, 7062, 6497, 4095, 3938, 9775,
            2354, 1524],
           dtype='int64')
train_ix
Int64Index([9836, 9612, 8342, 9370, 4083, 4934, 3687, 6377, 5909, 9585, 7639,
             203, 4761, 3864, 7409, 2606, 6429, 7062, 6497, 4095, 3938, 9775,
            1450,   54],
           dtype='int64')


KeyboardInterrupt: 

In [14]:
from common import compute_rmse_fraction

In [15]:
len(compute_rmse_fraction('hvac',s[range(4, 10)],'SanDiego')[0])

181

In [16]:
len(compute_rmse('hvac',s[range(4, 10)],'SanDiego')[0])

181

In [16]:
out = {}
for random_seed in range(2):
    out[random_seed] = {}
       
    for appliance in APPLIANCES_ORDER[1:]:
        out[random_seed][appliance] = {}
        for f in range(70, 110, 10):
            s = pd.concat(pred[random_seed][appliance][f]).ix[df.index]
            if appliance=="hvac":
                    out[random_seed][appliance][f] = compute_rmse_fraction(appliance,s[range(4, 10)],'SanDiego')[2]
            else:   
                out[random_seed][appliance][f] = compute_rmse_fraction(appliance, s,'SanDiego')[2]

    

In [25]:
means_df = pd.Panel(out).mean(axis=0)
means_df

,dw,fridge,hvac,mw,oven,wm
70,1.920209,7.010219,18.020753,1.727362,2.126271,0.730257
80,1.919044,6.863812,17.346531,1.766003,2.057335,0.737063
90,1.893030,6.974902,16.673514,1.749955,2.124033,0.749586
100,1.882632,7.051506,16.226711,1.759001,2.076530,0.673275


In [26]:
std_df = pd.Panel(out).std(axis=0)
std_df

,dw,fridge,hvac,mw,oven,wm
70,0.049394,0.393106,0.413556,0.003032,0.040199,0.052804
80,0.045225,0.315735,0.199118,0.017159,0.009605,0.070829
90,0.001614,0.007138,0.028247,0.000354,0.006826,0.015223
100,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
